In [68]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### 대화 내용을 기록하는 Agent

- `RunnableWithMessageHistory` 를 사용하여 인자로 `AgentExecutor` 를 넣어줍니다.

In [69]:
from langchain.agents import tool
from typing import List, Dict, Annotated

In [70]:
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL

In [71]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [72]:
from langchain_teddynote.messages import AgentStreamParser
from langchain_teddynote.messages import AgentCallbacks
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

### 도구 정의

In [73]:
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""

    print(f'검색어: {query}')

    news_tool = GoogleNews()

    return news_tool.search_by_keyword(query, k=5)

In [74]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

### Agent 프롬프트 작성

In [75]:
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (                                           
            "system",
            "You are a helpful assistant. "             
            "Make sure to use the `search_news` tool for searching keyword related news."
            "Be sure to use the 'python_repl_tool' tool when make python code."
            "Be sure to use the 'naver_news_crawl' tool when naver news crawlling."
        ),
        ("placeholder", "{chat_history}"),          # 이전 대화 내용을 넣을 곳을 잡아둔다.
        ("human", "{input}"),                       # 사용자 입력

        # 에이전트가 검색하는 과정들이나 내용 등을 끄적이는 메모장 같은 공간을 플레이스 홀더로 만들어준다
        ("placeholder", "{agent_scratchpad}"),       
    ]
)

In [76]:
tools = [search_keyword, python_repl_tool]

### Agent 생성

In [77]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

agent = create_tool_calling_agent(llm, tools, prompt)

### AgentExecutor

In [78]:
agent_executor = AgentExecutor(
    agent=agent,                    # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                    # agent 가 사용할 수 있는 도구 목록
    verbose=False,
    max_iterations=10,              # 최대 10번 까지 반복
    max_execution_time=10,          # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

### 중간 단계를 출력하는 사용자 정의 함수 정의

In [79]:
def tool_callback(tool) -> None:
    print(f'===== 도구 호출 =====')
    print(f"사용된 도구 이름 : {tool.get('tool')}")
    print(f'===== 도구 호출 =====')

In [80]:
def obervation_callback(observation) -> None:
    print(f'===== 관찰 내용 =====')
    print(f"Observation : {observation.get('observation')[0]}")
    print(f'===== 관찰 내용 =====')

In [81]:
def result_callback(result: str) -> None:
    print(f'===== 최종 답변 =====')
    print(result)
    print(f'===== 최종 답변 =====')

In [82]:
agent_callbacks = AgentCallbacks(
    tool_callback=tool_callback, 
    observation_callback=obervation_callback, 
    result_callback=result_callback
)

### AgentStreamParser

In [83]:
agent_stream_parser = AgentStreamParser(agent_callbacks)

In [84]:
store = {}

In [85]:
def get_session_history(session_ids):   # session_id 를 기반으로 세션 기록을 가져오는 함수    
    if session_ids not in store:        # session_id 가 store에 없는 경우        
        store[session_ids] = ChatMessageHistory()   # 새로운 ChatMessageHistory 객체를 딕셔너리에 저장
    
    return store[session_ids]           # 해당 세션 ID에 대한 세션 기록 리턴

In [86]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,                                    # session_id 를 기반으로 세션 기록을 가져오는 함수
    input_messages_key='input',                             # 프롬프트의 질문으로 들어가는 key: input
    history_messages_key='chat_history'                     # 프롬프트의 대화 기록이 들어가는 key: chat_history
)

In [87]:
response = agent_with_chat_history.stream(
    {'input': '안녕? 내 이름은 이인환이야.'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

In [88]:
for step in response:
    agent_stream_parser.process_agent_steps(step)

===== 최종 답변 =====
안녕하세요, 이인환님! 어떻게 도와드릴까요?
===== 최종 답변 =====


In [89]:
response = agent_with_chat_history.stream(
    {'input': '내 이름이 뭐라고?'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

In [90]:
response = agent_with_chat_history.stream(
    {'input': '나는 소프트웨어놀이터 라는 회사에 코딩강의 및 연구 개발을 하고있어'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

In [91]:
for step in response:
    agent_stream_parser.process_agent_steps(step)

===== 최종 답변 =====
소프트웨어놀이터에서 코딩 강의와 연구 개발을 하신다니 멋지네요! 어떤 주제에 대해 강의하시거나 연구하고 계신가요? 도움이 필요하신 부분이 있다면 말씀해 주세요.
===== 최종 답변 =====


In [92]:
response = agent_with_chat_history.stream(
    {
        'input': '랭체인에 대한 뉴스 5개를 검색해서 이메일 본문으로 작성해줘.'
        '수신인에는 `윤정 대리님` 그리고, 발신인에는 내 인적정보를 적어줘.'
        '정중한 어조로 작성하고, 메일의 시작과 끝에는 적절한 인사말과 맺음말을 적어줘.'
    }, 
    config={'configurable': {'session_id': 'abc123'}}
)

In [93]:
for step in response:
    agent_stream_parser.process_agent_steps(step)

===== 도구 호출 =====
사용된 도구 이름 : search_keyword
===== 도구 호출 =====
검색어: 랭체인
===== 관찰 내용 =====
Observation : {'url': 'https://news.google.com/rss/articles/CBMibkFVX3lxTE1qMC1EYjRZOEpMNVBEVVpSNm1BX3A4a3FrdmEyOTdDNWdjMGR1ZEgzWlpHZnRFZ1pSUXdKa3JnOXBQZU5qU3dKS3Q5VUNwLVZxRFZmbG1SZzJVSUoyM2lJbnZRbzhYYUJJcnFHWmp3?oc=5', 'content': '랭체인 LangChain 이란 무엇인가? | 인사이트리포트 | 삼성SDS - Samsung SDS'}
===== 관찰 내용 =====
===== 최종 답변 =====
아래는 "랭체인"에 대한 최근 뉴스 5개를 정리한 이메일 본문입니다.

---

**수신인:** 윤정 대리님  
**발신인:** 이인환

안녕하세요, 윤정 대리님.

최근 "랭체인"에 대한 흥미로운 뉴스들을 아래와 같이 정리해 보았습니다. 참고하시기 바랍니다.

1. [랭체인 LangChain 이란 무엇인가? | 인사이트리포트 | 삼성SDS](https://news.google.com/rss/articles/CBMibkFVX3lxTE1qMC1EYjRZOEpMNVBEVVpSNm1BX3A4a3FrdmEyOTdDNWdjMGR1ZEgzWlpHZnRFZ1pSUXdKa3JnOXBQZU5qU3dKS3Q5VUNwLVZxRFZmbG1SZzJVSUoyM2lJbnZRbzhYYUJJcnFHWmp3?oc=5)  
   - 랭체인에 대한 기본적인 개념과 활용 방안에 대해 설명합니다.

2. [개발자를 위한 랭체인 기초 개념과 환경 설정법 - 요즘IT](https://news.google.com/rss/articles/CBMiWkFVX3lxTFBtcm1OMmpJOUxvdnlkd0M0dVFfaGhma3BrTDRaVEY2OXFwd2tfUTZHVkFNSkZJYkZ

In [94]:
response = agent_with_chat_history.stream(
    {'input': '내 이름이 뭐라고?'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

In [95]:
for step in response:
    agent_stream_parser.process_agent_steps(step)

===== 최종 답변 =====
당신의 이름은 이인환입니다.
===== 최종 답변 =====


In [ ]:
result = agent_with_chat_history.stream(
    {'input': '좀전에 질문한 내용에 대해서 알려줘'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

i = 1

for step in result:
    print('==' * 30)
    # print(f'[step] ({i}) : \n', step)
    print(f"[step] ({i}): \n")

    if 'actions' in step:        
        print(f'===== [actions] 시작 =====')
        
        for action in step['actions']:
            print(f"action type: {type(action)}")
            print(action)
        print(f'===== [actions]  끝 =====')

        print(f'===== [messages] 시작 =====')

        for message in step['messages']:
            print(f"메시지 타입: {type(message).__name__}")
            print(f"내용 Message: {message.content}")

            if hasattr(message, 'tool_calls'):
                print('도구 호출 이름: ', message.tool_calls[0]['name'])
                print('도구 호출 인자: ', message.tool_calls[0]['args'])
                print('도구 호출 아이디: ', message.tool_calls[0]['id'])
                print('도구 호출 타입: ', message.tool_calls[0]['type'])
            else:
                print('도구 호출 없음')
        print(f'===== [messages] 끝 =====')
    
    elif 'steps' in step:        
        print(f'===== [steps] 시작 =====')

        for agent_step in step['steps']:
            print(f" 사용 도구 이름 agent_step.action.tool: {agent_step.action.tool}")
            print(f" 입력 쿼리 agent_step.action.tool_input: {agent_step.action.tool_input}")
            print(f" Log: {agent_step.action.log}")
            # print(f"(4) Observation: {agent_step.observation}")
            print(f" Observation: \n")

            if len(agent_step.observation)>0:
                for observation in agent_step.observation:
                    print(observation)

        print(f'===== [steps]  끝 =====')

        print(f'===== [messages] 시작 =====')
        for message in step['messages']:
            print(f"메시지 타입: {type(message).__name__}")
            print(f"내용 Message: {message.content}")

            if hasattr(message, 'tool_calls'):
                print('도구 호출 이름: ', message.tool_calls[0]['name'])
                print('도구 호출 인자: ', message.tool_calls[0]['args'])
                print('도구 호출 아이디: ', message.tool_calls[0]['id'])
                print('도구 호출 타입: ', message.tool_calls[0]['type'])
            else:
                print('도구 호출 없음')
        print(f'===== [messages] 끝 =====')

    
    elif 'output' in step:        
        print(f'===== [output] 시작 =====')
        print(f"[output]: {step['output']}")  
        print(f'===== [output]  끝 =====')

        print(f'===== [messages] 시작 =====')
        for message in step['messages']:
            print(f"메시지 타입: {type(message).__name__}")
            print(f"내용 Message: {message.content}")
        print(f'===== [messages] 끝 =====')

    i=i+1

[step] (1): 

===== [actions] 시작 =====
action type: <class 'langchain.agents.output_parsers.tools.ToolAgentAction'>
tool='search_keyword' tool_input={'query': '랭체인'} log="\nInvoking: `search_keyword` with `{'query': '랭체인'}`\n\n\n" message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Sy9yzMiu7xYCiMZSex1zPGef', 'function': {'arguments': '{"query":"랭체인"}', 'name': 'search_keyword'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac'}, id='run-9e6d5faf-1cbf-4d8b-bd62-1dece527d221', tool_calls=[{'name': 'search_keyword', 'args': {'query': '랭체인'}, 'id': 'call_Sy9yzMiu7xYCiMZSex1zPGef', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'search_keyword', 'args': '{"query":"랭체인"}', 'id': 'call_Sy9yzMiu7xYCiMZSex1zPGef', 'index': 0, 'type': 'tool_call_chunk'}])] tool_call_id='call_Sy9yzMiu7xYCiMZSex1zPGef'
===== [actions]  끝 =====
===== [messages] 시작 =

In [ ]:
result = agent_with_chat_history.stream(
    {'input': '윤정 대리님께 보내기 위해 작성한 이메일 내용을 알려줘'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

i = 1

for step in result:
    print('==' * 30)
    # print(f'[step] ({i}) : \n', step)
    print(f"[step] ({i}): \n")

    if 'actions' in step:        
        print(f'===== [actions] 시작 =====')
        
        for action in step['actions']:
            print(f"action type: {type(action)}")
            print(action)
        print(f'===== [actions]  끝 =====')

        print(f'===== [messages] 시작 =====')

        for message in step['messages']:
            print(f"메시지 타입: {type(message).__name__}")
            print(f"내용 Message: {message.content}")

            if hasattr(message, 'tool_calls'):
                print('도구 호출 이름: ', message.tool_calls[0]['name'])
                print('도구 호출 인자: ', message.tool_calls[0]['args'])
                print('도구 호출 아이디: ', message.tool_calls[0]['id'])
                print('도구 호출 타입: ', message.tool_calls[0]['type'])
            else:
                print('도구 호출 없음')
        print(f'===== [messages] 끝 =====')
    
    elif 'steps' in step:        
        print(f'===== [steps] 시작 =====')

        for agent_step in step['steps']:
            print(f" 사용 도구 이름 agent_step.action.tool: {agent_step.action.tool}")
            print(f" 입력 쿼리 agent_step.action.tool_input: {agent_step.action.tool_input}")
            print(f" Log: {agent_step.action.log}")
            # print(f"(4) Observation: {agent_step.observation}")
            print(f" Observation: \n")

            if len(agent_step.observation)>0:
                for observation in agent_step.observation:
                    print(observation)

        print(f'===== [steps]  끝 =====')

        print(f'===== [messages] 시작 =====')
        for message in step['messages']:
            print(f"메시지 타입: {type(message).__name__}")
            print(f"내용 Message: {message.content}")

            if hasattr(message, 'tool_calls'):
                print('도구 호출 이름: ', message.tool_calls[0]['name'])
                print('도구 호출 인자: ', message.tool_calls[0]['args'])
                print('도구 호출 아이디: ', message.tool_calls[0]['id'])
                print('도구 호출 타입: ', message.tool_calls[0]['type'])
            else:
                print('도구 호출 없음')
        print(f'===== [messages] 끝 =====')

    
    elif 'output' in step:        
        print(f'===== [output] 시작 =====')
        print(f"[output]: {step['output']}")  
        print(f'===== [output]  끝 =====')

        print(f'===== [messages] 시작 =====')
        for message in step['messages']:
            print(f"메시지 타입: {type(message).__name__}")
            print(f"내용 Message: {message.content}")
        print(f'===== [messages] 끝 =====')

    i=i+1

[step] (1): 

===== [output] 시작 =====
[output]: 아래는 윤정 대리님께 보내기 위해 작성한 이메일 내용입니다:

---

**수신인:** 윤정 대리님  
**발신인:** 이인환

안녕하세요, 윤정 대리님.

최근 "랭체인"에 대한 흥미로운 뉴스들을 아래와 같이 정리해 보았습니다. 참고하시기 바랍니다.

1. **[랭체인 LangChain 이란 무엇인가? | 인사이트리포트 | 삼성SDS](https://news.google.com/rss/articles/CBMibkFVX3lxTE1qMC1EYjRZOEpMNVBEVVpSNm1BX3A4a3FrdmEyOTdDNWdjMGR1ZEgzWlpHZnRFZ1pSUXdKa3JnOXBQZU5qU3dKS3Q5VUNwLVZxRFZmbG1SZzJVSUoyM2lJbnZRbzhYYUJJcnFHWmp3?oc=5)**  
   - 랭체인에 대한 기본적인 개념과 활용 방안에 대해 설명합니다.

2. **[개발자를 위한 랭체인 기초 개념과 환경 설정법 - 요즘IT](https://news.google.com/rss/articles/CBMiWkFVX3lxTFBtcm1OMmpJOUxvdnlkd0M0dVFfaGhma3BrTDRaVEY2OXFwd2tfUTZHVkFNSkZJYkZLc0x2dzFWWDNldF9TaVhIU0NxSUxyMlpIRC1KNksyLWdNQQ?oc=5)**  
   - 랭체인을 시작하는 개발자들을 위한 기초 개념과 환경 설정 방법을 다룹니다.

3. **[20화 19. RAG(검색증강생성) 지원 - 브런치](https://news.google.com/rss/articles/CBMiRkFVX3lxTE04LVVxZF8wc09QVGxFWjRURWktdTdRNE5VYWYweVkteHdxaDRDbjY1OUpLeDUtbWxTVnRWT3gwTWVsSk5OUHc?oc=5)**  
   - 랭체인에서의 검색 증강 생성(RAG) 지원에 대한 내용을 소개합니다.

4. **[구글, 데이터베이스용 ‘생성형 AI 툴박스’ 공